# Break-even price and power purchase agreement
HyDesign calculates the price of electricity and/or hydrogen that would break-even i.e. result in NPV of zero. These results are based on the current hybrid configuration and operation. This means that the results will vary depending on which input parameters you are using, but will generally converge when the input price parameters are approaching the resulting break-even prices.

## Break-even price of H2
The break-even price of H2 can be obtained as following.

In [ ]:
# Detect if running in Kaggle
import os 
if os.environ.get('KAGGLE_KERNEL_RUN_TYPE'):
    mypaths = !python -c "import sysconfig; print(sysconfig.get_paths()['purelib'])"
    mypath = mypaths[0]
    !pip install trash-cli
    !trash $mypath/numpy*
    !pip install --upgrade numpy
    !pip install finitediff
    import os
    os.kill(os.getpid(), 9)

In [ ]:
# Install hydesign if needed
import importlib
if not importlib.util.find_spec("hydesign"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git   

In [1]:
%%capture
import pandas as pd
from hydesign.hpp_assembly import hpp_model
from hydesign.hpp_assembly_P2X import hpp_model_P2X
from hydesign.examples import examples_filepath

In [2]:
examples_sites = pd.read_csv(f'{examples_filepath}examples_sites.csv', index_col=0)
name = 'France_good_wind'
ex_site = examples_sites.loc[examples_sites.name == name]
longitude = ex_site['longitude'].values[0]
latitude = ex_site['latitude'].values[0]
altitude = ex_site['altitude'].values[0]
input_ts_fn = examples_filepath+ex_site['input_ts_fn'].values[0]
sim_pars_fn = examples_filepath+ex_site['sim_pars_fn'].values[0]
H2_demand_fn = examples_filepath+ex_site['H2_demand_col'].values[0]
hpp = hpp_model_P2X(
    latitude,
    longitude,
    altitude,
    num_batteries = 1,
    work_dir = './',
    sim_pars_fn = sim_pars_fn,
    input_ts_fn = input_ts_fn,
    H2_demand_fn=H2_demand_fn) 



Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302.0


In [3]:
x = [10.0, 350.0, 5.0, 20.0, 7.0, 100.0, 50.0, 180.0, 1.5, 0.0, 3.0, 5.0, 100.0, 2500.0]
outs = hpp.evaluate(*x)

In [4]:
hpp.print_design(x, outs)


Design:
---------------
clearance [m]: 10.000
sp [W/m2]: 350.000
p_rated [MW]: 5.000
Nwt: 20.000
wind_MW_per_km2 [MW/km2]: 7.000
solar_MW [MW]: 100.000
surface_tilt [deg]: 50.000
surface_azimuth [deg]: 180.000
DC_AC_ratio: 1.500
b_P [MW]: 0.000
b_E_h [h]: 3.000
cost_of_battery_P_fluct_in_peak_price_ratio: 5.000
ptg_MW [MW]: 100.000
HSS_kg [kg]: 2500.000


NPV_over_CAPEX: 0.439
NPV [MEuro]: 93.705
IRR: 0.101
LCOE [Euro/MWh]: 27.117
LCOH [Euro/kg]: 4.896
Revenue [MEuro]: 852.223
CAPEX [MEuro]: 213.615
OPEX [MEuro]: 3.819
penalty lifetime [MEuro]: 0.000
AEP [GWh]: 538.930
annual_Power2Grid [GWh]: 59.243
GUF: 0.205
annual_H2 [tons]: 6826.706
annual_P_ptg [GWh]: 479.686
grid [MW]: 300.000
wind [MW]: 100.000
solar [MW]: 100.000
PtG [MW]: 100.000
HSS [kg]: 2500.000
Battery Energy [MWh]: 0.000
Battery Power [MW]: 0.000
Total curtailment [GWh]: 0.000
Awpp [km2]: 14.286
Apvp [km2]: 1.226
Rotor diam [m]: 134.867
Hub height [m]: 77.434
Number of batteries used in lifetime: 0.000
Break-even H2 pri

The break-even H2 price is seen in the above table and can also be extracted directly:

In [5]:
print(f'Input H2 price: {float(hpp.prob["price_H2"]):.2f}')
print(f'Break-even H2 price: {float(hpp.prob["break_even_H2_price"]):.2f}')

Input H2 price: 5.00
Break-even H2 price: 3.65


## Power Purchase Agreement (PPA)
Constant price can be introduced by supplying a time series of constant prices or setting the PPA price directly when instantiating the hpp model:

In [6]:
PPA = 40 # Euro/MWh
hpp = hpp_model(
    latitude,
    longitude,
    altitude,
    num_batteries = 1,
    work_dir = './',
    sim_pars_fn = sim_pars_fn,
    input_ts_fn = input_ts_fn,
    ppa_price=PPA,)


Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302.0


In [7]:
x = [10.0, 350.0, 5.0, 20.0, 7.0, 100.0, 50.0, 180.0, 1.5, 0.0, 3.0, 5.0]
outs = hpp.evaluate(*x)

In [8]:
hpp.print_design(x, outs)


Design:
---------------
clearance [m]: 10.000
sp [W/m2]: 350.000
p_rated [MW]: 5.000
Nwt: 20.000
wind_MW_per_km2 [MW/km2]: 7.000
solar_MW [MW]: 100.000
surface_tilt [deg]: 50.000
surface_azimuth [deg]: 180.000
DC_AC_ratio: 1.500
b_P [MW]: 0.000
b_E_h [h]: 3.000
cost_of_battery_P_fluct_in_peak_price_ratio: 5.000


NPV_over_CAPEX: 0.080
NPV [MEuro]: 13.205
IRR: 0.062
LCOE [Euro/MWh]: 30.306
CAPEX [MEuro]: 164.865
OPEX [MEuro]: 2.466
Wind CAPEX [MEuro]: 76.430
Wind OPEX [MEuro]: 1.791
PV CAPEX [MEuro]: 33.167
PV OPEX [MEuro]: 0.675
Batt CAPEX [MEuro]: 0.000
Batt OPEX [MEuro]: 0.000
Shared CAPEX [MEuro]: 55.268
Shared Opex [MEuro]: 0.000
penalty lifetime [MEuro]: 0.000
AEP [GWh]: 467.714
GUF: 0.178
grid [MW]: 300.000
wind [MW]: 100.000
solar [MW]: 100.000
Battery Energy [MWh]: 0.000
Battery Power [MW]: 0.000
Total curtailment [GWh]: 0.000
Awpp [km2]: 14.286
Apvp [km2]: 1.226
Plant area [km2]: 14.286
Rotor diam [m]: 134.867
Hub height [m]: 77.434
Number of batteries used in lifetime: 0.000

The break-even PPA price is seen in the above table and can also be extracted directly:

In [9]:
print(f'Input H2 price: {float(PPA):.2f}')
print(f'Break-even H2 price: {float(hpp.prob["break_even_PPA_price"]):.2f}')

Input H2 price: 40.00
Break-even H2 price: 37.41


LCOE is a measure of lifetime cost of producing power, but comes out somewhat different from the break-even price. This is because break-even price is based on NPV calculations that takes into account e.g. the corporate tax rate as well as penalties incurred from not meeting expected loads.